In [45]:
import nltk

In [46]:
from nltk.corpus import genesis

In [47]:
genesis_words = genesis.words()

In [48]:
def filter_letter(let):

    if ord('a') <= ord(let) <= ord('z'):
        return True
    
    else:
        return False

In [49]:
def make_blocks(word_list,block_size):
    word_str = "".join(word_list)
    new_str = ""
    blocked_arr = []



    for i in range(0,len(word_str)):
        
        let = word_str[i].lower()
        if filter_letter(let):
            new_str += let
        else:
            pass
    
    del word_str

    for i in range(0, len(new_str), block_size):
        block = new_str[i:i+block_size]
        
        if len(block) == block_size:
            blocked_arr.append(block)
        else:
            diff = block_size - len(block)
            padded_block = block + "x" * diff
            blocked_arr.append(padded_block)

    return blocked_arr

In [50]:
pt_blocks = make_blocks(genesis_words,8)

print(pt_blocks[:10])

['inthebeg', 'inninggo', 'dcreated', 'theheave', 'nandthee', 'arthandt', 'heearthw', 'aswithou', 'tformand', 'voidandd']


In [51]:
def let2num(let):
    return ord(let.lower()) - ord('a')

def num2let(num):
    return chr(num + ord('a'))


def caesar_encrypt(let,key):

    n = let2num(let)
    c_n = (n + key) % 26
    ct = num2let(c_n)
    
    return ct


In [52]:
caesar_encrypt('a',2)

'c'

In [53]:
ct_blocks = []
# key_blocks = []

# for key in range(0,25):

FIXED_KEY = 5    
for idx in range(len(pt_blocks)):
    
    ct_t = ""   
    

    # encrypt the block
    for let in pt_blocks[idx]:
        ct_t += caesar_encrypt(let,FIXED_KEY)

    ct_blocks.append(ct_t)

In [54]:
print(len(ct_blocks))

134339


In [55]:
print(len(pt_blocks))

134339


In [56]:
pt_blocks[:5]

['inthebeg', 'inninggo', 'dcreated', 'theheave', 'nandthee']

In [57]:
ct_blocks[:5]

['nsymjgjl', 'nssnsllt', 'ihwjfyji', 'ymjmjfaj', 'sfsiymjj']

In [58]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense,InputLayer

In [59]:
batch_size = 64
num_samples = 137957
latent_dim = 256
epochs = 15

In [60]:
input_texts = pt_blocks
target_texts = ct_blocks

In [61]:
# Start token \t
# end token \n

input_characters = set()
target_characters = set()

for i in range(len(input_texts)):

    input_texts[i] = input_texts[i] + '\n'
    
    for char in input_texts[i]:
        if char not in input_characters:
            input_characters.add(char)



for j in range(len(target_texts)):
    target_texts[j] = '\t' + target_texts[j] + '\n'
    
    for char in target_texts[j]:
        if char not in target_characters:
            target_characters.add(char)

In [62]:
unique_chars = set()

for word in pt_blocks:

    for let in word:

        if not let in unique_chars:
            unique_chars.add(let)


print(len(unique_chars))

27


In [63]:
input_characters.add(' ')
target_characters.add(' ')
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])


print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 134339
Number of unique input tokens: 28
Number of unique output tokens: 29
Max sequence length for inputs: 9
Max sequence length for outputs: 10


In [64]:
input_token_index = dict([(char, i)
                         for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i)
                          for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype="float32",
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype="float32",
)

In [65]:
encoder_input_data.shape

(134339, 9, 28)

In [66]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1:, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1:, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [67]:
print(encoder_input_data.shape)
print(decoder_target_data.shape)

(134339, 9, 28)
(134339, 10, 29)


In [68]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(
    latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(
    decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [69]:
print(encoder_input_data.shape)
print(num_encoder_tokens)

(134339, 9, 28)
28


In [70]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s_vig_model.keras")

Epoch 1/15
1680/1680 [==============================] - 82s 46ms/step - loss: 0.9574 - accuracy: 0.7122 - val_loss: 0.3355 - val_accuracy: 0.8929
Epoch 2/15
1680/1680 [==============================] - 73s 43ms/step - loss: 0.0999 - accuracy: 0.9728 - val_loss: 0.2031 - val_accuracy: 0.9437
Epoch 3/15
1680/1680 [==============================] - 61s 36ms/step - loss: 0.0376 - accuracy: 0.9906 - val_loss: 0.0388 - val_accuracy: 0.9888
Epoch 4/15
1680/1680 [==============================] - 62s 37ms/step - loss: 0.0197 - accuracy: 0.9952 - val_loss: 0.1265 - val_accuracy: 0.9731
Epoch 5/15
1680/1680 [==============================] - 61s 36ms/step - loss: 0.0127 - accuracy: 0.9970 - val_loss: 0.0365 - val_accuracy: 0.9896
Epoch 6/15
1680/1680 [==============================] - 60s 35ms/step - loss: 0.0089 - accuracy: 0.9980 - val_loss: 0.0141 - val_accuracy: 0.9961
Epoch 7/15
1680/1680 [==============================] - 60s 36ms/step - loss: 0.0066 - accuracy: 0.9986 - val_loss: 0.0621 -